# Задание 1

Сделать класс матрицы, в котором определить операции сложения и умножения (матричного и покомпонентного) через перегрузку операторов `+, *, @` (как в `numpy`). Вызывать исключения, если матрицы на входе некорректной размерности (`ValueError`)
Сгенерировать две матрицы через `np.random.randint(0, 10, (10, 10))` c seed-ом 0 и над ними провести все три операции. Записать результаты в текстовые файлы, названные `matrix+.txt`,
`matrix*.txt`,
`matrix@.txt`, соответственно. Это будет артефактом задачи.


In [ ]:
import numpy as np

np.random.seed(0)

In [ ]:
type(9.)==float

In [ ]:
import copy

class Matrix():

    @staticmethod
    def matrix_shape(matrix):
        return (len(matrix),len(matrix[0]))

    def __init__(self, mas):
        self.matrix = list(map(list, mas))
        self.matrix_shape = self.matrix_shape(self.matrix)

    def __add__(self, other):
        types = [int,float]
        sum_matrix = copy.deepcopy(self.matrix)
        if type(other) in types:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    sum_matrix[i][j]+=other
        elif not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix or Number')
        elif self.matrix_shape != other.matrix_shape:
            raise ValueError(
                f'Expected size {other.matrix_shape!r} not equal sizes {self.matrix_shape!r} ')
        else:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    sum_matrix[i][j]+=other.matrix[i][j]
        return sum_matrix

    def __mul__(self, other):
        types = [int,float]
        mult_matrix = copy.deepcopy(self.matrix)
        if type(other) in types:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    mult_matrix[i][j]*=other
        elif not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix or Number')
        elif self.matrix_shape != other.matrix_shape:
            raise ValueError(
                f'Expected size {other.matrix_shape!r} not equal sizes {self.matrix_shape!r} ')
        else:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    mult_matrix[i][j]*=other.matrix[i][j]
        return mult_matrix

    def __matmul__(self, other):
        if not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix')
        elif self.matrix_shape[1] != other.matrix_shape[0]:
            raise ValueError(
                f'Expected size {other.matrix_shape[0]!r} not equal sizes {self.matrix_shape[1]!r} ')
        else:
            mult_matrix = [[0 for i in range(other.matrix_shape[1])] for j in range(self.matrix_shape[0])]
            for i in range(self.matrix_shape[0]):
                for j in range(other.matrix_shape[1]):
                    mult_matrix[i][j]= sum([self.matrix[i][k]*other.matrix[k][j] for k in range(other.matrix_shape[0])])
        return mult_matrix

In [ ]:
a = Matrix(np.random.randint(0, 10, (10, 10)))
b = Matrix(np.random.randint(0, 10, (10, 10)))

In [ ]:
a.matrix

In [ ]:
b.matrix

In [ ]:
a+b

In [ ]:
a*b

In [ ]:
a@b

# Задача 2

Используя примеси numpy, сделать класс, который будет уметь выполнять все стандартные арифметические операции.
Также добавить через примеси: запись объекта в файл, красивое отображение в консоли (`__str__`), getter-ы и setter-ы для полей класса
В самих классах должно быть минимальное количество методов
Артефакты задачи - аналогично задаче 3.1


In [ ]:
import numpy as np
from numpy.lib.mixins import NDArrayOperatorsMixin


class StrMixin:
    def __str__(self):
        return '\n'.join(['\t'.join(map(str, row)) for row in self.matrix])


class SaveMixin:
    def save(self, filename):
        np.savetxt(filename, self.matrix, fmt='%d', delimiter="\t")


class SetGetMixin:
    @property
    def data(self):
        return self._matrix

    @data.setter
    def data(self, matrix):
        self.matrix = matrix
        if len(matrix) == 0:
            raise ValueError('Empty matrices are not supported')
        if len(matrix[0]) == 0:
            raise ValueError('Empty columns are not supported')
        self.shape = (len(matrix),len(matrix[0]))


class MixinMatrix(NDArrayOperatorsMixin, StrMixin, SaveMixin, SetGetMixin):
    def __init__(self, matrix):
        self.data = np.asarray(matrix)

    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        new_inputs = []
        for input_ in inputs:
            if isinstance(input_, MixinMatrix):
                new_inputs.append(input_.matrix)
            else:
                new_inputs.append(input_)

        result = getattr(ufunc, method)(*new_inputs, **kwargs)

        if method == '__call__' and isinstance(result, np.ndarray):
            return MixinMatrix(result.tolist())
        else:
            return result

In [ ]:
# a = [[11,2,3],[4,5,6],[4,5,6]]
# b = [[1,2,3],[4,5,6],[4,5,6]]
a = np.random.randint(0, 10, (10, 10))
b =np.random.randint(0, 10, (10, 10))

A = MixinMatrix(a)
B = MixinMatrix(b)

(A + B).save('matrix+.txt')
(A * B).save('matrix*.txt')
(A @ B).save('matrix@.txt')

In [ ]:
A.shape

In [ ]:
A.matrix

In [ ]:
print(A)

In [ ]:
(A + B).matrix

In [ ]:
(A * B).matrix

In [ ]:
(A @ B).matrix

In [ ]:
sum(sum(row) for row in ((1,2,3),(4,5,6),(4,5,6))) % 7247

# Задача 3

Задача является продолжением задачи 3.1
Придумать и реализовать простейшую хэш-функцию (дать краткое текстовое описание в комментариях в коде) для матрицы  в методе __hash__ (вынести в примесь).
Ограничение на хэш-функцию - она должна быть не константой (не возвращать всегда одно число)
Настроить кэширование произведения матриц по этой хэш-функции
Найти коллизию в хэш-функции (если поиск производится кодом, то код также нужно выложить)
Артефакт - 7 файлов.
`A.txt, B.txt, C.txt, D.txt` - матрицы, такие, что
`(hash(A) == hash(C)) and (A != C) and (B == D) and (A @ B != C @ D)`

`AB.txt` - результат произведения A @ B

`CD.txt` - настоящий результат произведения C @ D

`hash.txt` - хэш матриц AB и CD


In [ ]:
class HashMixin:
    # Наверное, самое простое, что мы можем сделать – это взять и вычислить остаток от деления
    # Для большей эффективности возьмём простоё число, для боьшого объёма таблицы - большое число
    def __hash__(self):
        matrix, s = tuple(map(tuple, self.matrix)), 0
        for row in matrix:
            s+=sum(row)
        return int(s % 7247)

class Matrix(HashMixin):

    @staticmethod
    def matrix_shape(matrix):
        return (len(matrix),len(matrix[0]))

    def __init__(self, mas):
        self.matrix = list(map(list, mas))
        self.matrix_shape = self.matrix_shape(self.matrix)

    def __add__(self, other):
        types = [int,float]
        sum_matrix = copy.deepcopy(self.matrix)
        if type(other) in types:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    sum_matrix[i][j]+=other
        elif not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix or Number')
        elif self.matrix_shape != other.matrix_shape:
            raise ValueError(
                f'Expected size {other.matrix_shape!r} not equal sizes {self.matrix_shape!r} ')
        else:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    sum_matrix[i][j]+=other.matrix[i][j]
        return sum_matrix

    def __mul__(self, other):
        types = [int,float]
        mult_matrix = copy.deepcopy(self.matrix)
        if type(other) in types:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    mult_matrix[i][j]*=other
        elif not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix or Number')
        elif self.matrix_shape != other.matrix_shape:
            raise ValueError(
                f'Expected size {other.matrix_shape!r} not equal sizes {self.matrix_shape!r} ')
        else:
            for i in range(self.matrix_shape[0]):
                for j in range(self.matrix_shape[1]):
                    mult_matrix[i][j]*=other.matrix[i][j]
        return mult_matrix

    def __matmul__(self, other):
        if not isinstance(other, type(self)):
             raise TypeError(
                f'Expected {type(other)!r} to be an Matrix')
        elif self.matrix_shape[1] != other.matrix_shape[0]:
            raise ValueError(
                f'Expected size {other.matrix_shape[0]!r} not equal sizes {self.matrix_shape[1]!r} ')
        else:
            mult_matrix = [[0 for i in range(other.matrix_shape[1])] for j in range(self.matrix_shape[0])]
            for i in range(self.matrix_shape[0]):
                for j in range(other.matrix_shape[1]):
                    mult_matrix[i][j]= sum([self.matrix[i][k]*other.matrix[k][j] for k in range(other.matrix_shape[0])])
        return mult_matrix

In [ ]:
a = Matrix([[1,2,3],[2,3,1],[3,1,2]])
c = Matrix([[1,1,1],[2,2,2],[3,3,3]])
b = Matrix([[1,0,0],[0,1,1],[0,0,1]])
d = Matrix([[1,0,0],[0,1,1],[0,0,1]])

In [ ]:
a@b

In [ ]:
hash(a)

In [ ]:
hash(c)

In [ ]:
hash(d)

In [ ]:
hash(b)

In [ ]:
hash(Matrix(a@b))

In [ ]:
hash(Matrix(c@d))